In [74]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 环境变量

In [75]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

## 模型加载

In [76]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    # embedding = OpenAIEmbeddings(deployment = os.environ.get('DEPLOYMENT_NAME_EMBEDDING'),chunk_size=1)
    
    return llm,chat

llm,chat = load_azureLLM()

## Chain

In [77]:

from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate


person_info = """{
    "gender": "女",
    "age": "20岁",
    "height": "150cm",
    "weight": "50kg"
}"""


fitness_data = """{
    "action": "深蹲",
    "timing": "60秒",
    "count_type": "次数",
    "count_result": "30次"
}"""

### 顺序链

#### 动作评估链

In [78]:

evaluate_standards = """评估维度：\
1. 力量：指肌肉在运动中产生的最大力量。常见的力量训练包括举重、俯卧撑、深蹲等。通过计次评估。\
2. 速度：指肌肉完成一定距离或动作所需时间的能力。常见的速度训练包括快速小跑、跳绳等。通过计时评估。\
3. 耐力：指肌肉长时间持续工作的能力。常见的耐力训练包括深蹲跳、高抬腿等。通过计次或计时评估。\
4. 灵敏：指肌肉对刺激的快速反应能力。常见的灵敏度训练包括跳绳、跳跃、击剑等。通过计次评估。\
5. 协调：指身体各部位肌肉协调运动的能力，常见的协调性训练包括瑜伽、桑巴、舞蹈等。通过计时评估。\
6. 稳定：指骨骼及肌肉在身体静止或运动状态下维持平衡的能力。常见的稳定性训练包括平板支撑、单脚站立等。通过计时评估。\
评估方法：\
根据动作的计次和计时数据，严格评估动作在每个维度的得分。\
评估得分，最低0分，最高10分。如果无法评估，则为0分。
"""


evaluate_prompt = ChatPromptTemplate.from_template(
    template='您是资深的健身运动专家，请参考用户的个人信息，根据运动评估标准，严格以对用户的运动数据进行评估。\
        运动评估标准是：{standards},\
        用户的个人信息是：{person},\
        用户的运动数据是：{data}'
    )


chain_evaluate = LLMChain(llm=llm, prompt=evaluate_prompt,output_key="evaluate_result")

#### 制定计划链

In [79]:

plan_prompt = ChatPromptTemplate.from_template(
    template="您是资深的健身运动专家，根据用户的个人信息，以及运动评价结果，以健身计划制定标准为原则，为用户制定个性化的健身计划。\
        用户的个人信息是：{person},\
        用户的运动评价结果是：{evaluate_result}"
    )
chain_plan = LLMChain(llm=llm, prompt=plan_prompt,output_key="plan_reslut")


#### 构建顺序链

In [80]:

chain = SequentialChain(chains=[chain_evaluate,chain_plan],
                        input_variables=["standards","person","data"],
                        output_variables=["evaluate_result","plan_reslut"],
                        verbose=True)

#### 执行顺序链

In [81]:

chain({"standards":evaluate_standards,"person":person_info,"data":fitness_data})



> Entering new SequentialChain chain...

> Finished chain.


{'standards': '评估维度：1. 力量：指肌肉在运动中产生的最大力量。常见的力量训练包括举重、俯卧撑、深蹲等。通过计次评估。2. 速度：指肌肉完成一定距离或动作所需时间的能力。常见的速度训练包括快速小跑、跳绳等。通过计时评估。3. 耐力：指肌肉长时间持续工作的能力。常见的耐力训练包括深蹲跳、高抬腿等。通过计次或计时评估。4. 灵敏：指肌肉对刺激的快速反应能力。常见的灵敏度训练包括跳绳、跳跃、击剑等。通过计次评估。5. 协调：指身体各部位肌肉协调运动的能力，常见的协调性训练包括瑜伽、桑巴、舞蹈等。通过计时评估。6. 稳定：指骨骼及肌肉在身体静止或运动状态下维持平衡的能力。常见的稳定性训练包括平板支撑、单脚站立等。通过计时评估。评估方法：根据动作的计次和计时数据，严格评估动作在每个维度的得分。评估得分，最低0分，最高10分。如果无法评估，则为0分。\n',
 'person': '{\n    "gender": "女",\n    "age": "20岁",\n    "height": "150cm",\n    "weight": "50kg"\n}',
 'data': '{\n    "action": "深蹲",\n    "timing": "60秒",\n    "count_type": "次数",\n    "count_result": "30次"\n}',
 'evaluate_result': '\n\n答：根据用户的个人信息和运动数据，我们可以对用户的力量进行评估。根据用户的运动数据，用户在60秒内完成30次深蹲，因此可以得出用户的力量得分为10分。',
 'plan_reslut': '根据用户的个人信息，我们可以制定一个适合用户的健身计划，包括：\n\n1. 力量训练：每周至少进行3次力量训练，每次训练时间不少于30分钟，每次训练的重量应适当增加，以提高用户的力量水平。\n\n2. 肌肉练习：每周至少进行3次肌肉练习，每次训练时间不少于30分钟，每次训练的重量应适当增加，以提高用户的肌肉素质。\n\n3. 有氧运动：每周至少进行3次有氧运动，每次训练时间不少于30分钟，每次训练的强度应适当增加，以提高用户的有氧能力。\n\n4. 休息：每周至少休息2天，以保证用户的身体和精神得到充分休息。'}